In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

Using TensorFlow backend.


In [2]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [5]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [6]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [7]:
samples = samples.reshape(nos,9*9*1920)

In [8]:
del X
del y

In [9]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [10]:
pbounds = {
    'epochs':(int(30),int(100)),
    'learning_rate':(1e-4, 1e-2),
    'batch_size':(8,30)
}

In [11]:
def evaluate_network(epochs, learning_rate, batch_size):
  accuracies = []
  f = 1

  for train_index, test_index in skf.split(samples, labels):
    X_train = samples[train_index]
    X_test = samples[test_index]
    y_train = labels[train_index]
    y_test = labels[test_index]

    model = models.Sequential()
    model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = optimizers.Adam(lr = learning_rate),loss = 'binary_crossentropy',metrics = ['accuracy'])
    my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

    print('Fold {}'.format(f))
    history = model.fit(X_train, y_train, epochs = int(epochs), batch_size = int(batch_size), validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

    test_preds = model.predict(X_test, batch_size = 20)
    test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

    accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
    f+=1


  zero_acc = 0
  one_acc = 0

  for i in accuracies:
    zero_acc+= i[0]
    one_acc+=i[1]

  zero_acc/=8
  one_acc/=8

  print("Accuracies: ",accuracies)
  return (zero_acc+one_acc)


In [12]:
# !pip3 install bayesian-optimization

In [13]:
from bayes_opt import BayesianOptimization

In [14]:
optimizer = BayesianOptimization(
    f = evaluate_network,
    pbounds = pbounds,
    verbose = 2
)

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=2,
)

|   iter    |  target   | batch_... |  epochs   | learni... |
-------------------------------------------------------------
Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/85
413/413 [==============================] - 10s 24ms/step - loss: 14.7038 - accuracy: 0.6586 - val_loss: 7.7002 - val_accuracy: 0.7119
Epoch 2/85
413/413 [==============================] - 9s 23ms/step - loss: 4.0958 - accuracy: 0.8305 - val_loss: 1.2075 - val_accuracy: 0.8814
Epoch 3/85
413/413 [==============================] - 9s 23ms/step - loss: 1.8326 - accuracy: 0.8741 - val_loss: 3.8054 - val_accuracy: 0.7288
Epoch 4/85
413/413 [==============================] - 9s 23ms/step - loss: 0.9581 - accuracy: 0.9274 - val_loss: 1.8193 - val_accuracy: 0.7966
Epoch 5/85
413/413 [==============================] - 10s 23ms/step - loss: 0.5792 - accuracy: 0.9225 - val_loss: 1.9539 - val_accuracy: 0.7797
Epoch 6/85
413/413 [==============================] - 9s 23ms/step - loss: 0.7050 - accuracy: 0.9516 - val

In [ ]:
print(optimizer.max)